# Week 01 - Lesson 02 - Building Chatbots with LangChain

## Overview

This notebook builds upon Lesson 01 by teaching you how to create functional chatbots using LangChain. You'll learn how to build conversational AI systems that can maintain context, manage memory, and provide engaging user experiences.

### Learning Objectives
By the end of this notebook, you will:
1. Understand the fundamentals of chatbot architecture in LangChain
2. Learn how to implement conversation memory and state management
3. Master prompt templates for chatbot interactions
4. Implement streaming responses for better user experience
5. Understand enterprise chatbot deployment considerations

## Requirements

- **Python** installed (https://www.python.org/downloads/)
- **Jupyter Notebook** (already installed in this environment)
- **OpenAI API Key** (https://platform.openai.com/api-keys)
- **LangSmith Account** (optional, for tracing and debugging)
- The following Python libraries:
  - `langchain` (for chatbot orchestration)
  - `langchain-openai` (for OpenAI integration)
  - `langgraph` (for workflow management)
  - `pydantic` (for data validation)

## Context

Chatbots represent the next evolution of Chat Models, transforming them into interactive, conversational systems. They serve as:

- **Customer Service Interfaces**: Automated support with human escalation capabilities
- **Business Process Automation**: Intelligent workflow orchestration
- **Knowledge Management Systems**: Dynamic information retrieval and synthesis
- **User Experience Enhancers**: Personalized, context-aware interactions

Understanding chatbot development is crucial for:
- Building customer support systems
- Creating internal knowledge assistants
- Developing sales and marketing automation
- Implementing intelligent business process workflows

## 1. Environment Setup and Dependencies

First, let's ensure we have all necessary dependencies installed and configured.

In [1]:
# Install required dependencies
%pip install -U --quiet langchain langchain-openai langgraph pydantic

Note: you may need to restart the kernel to use updated packages.


In [2]:
# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

# LangGraph imports for chatbot workflow
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver

print("✅ All dependencies imported successfully!")

✅ All dependencies imported successfully!


## 2. Understanding Chatbot Architecture in LangChain

Before diving into code, let's understand the fundamental concepts of chatbot development:

### Core Components

1. **State Management**: Tracks conversation context and user information
2. **Message History**: Maintains conversation flow and context
3. **Prompt Templates**: Define chatbot behavior and personality
4. **Workflow Engine**: Manages the flow of conversation processing
5. **Memory System**: Persists conversation state across sessions

## 3. Setting Up the Foundation

Let's start by setting up the basic components needed for our chatbot:

In [21]:
# Your OpenAI API Key here
OPENAI_API_KEY = "sk-your-openai-api-key"

In [4]:
# Initialize the Chat Model
# This will be the core intelligence of our chatbot
model = ChatOpenAI(
    model="gpt-4o-mini",  # The specific model to use
    api_key=OPENAI_API_KEY,
    temperature=0.7,  # Slightly creative for conversational responses
    max_tokens=1000   # Reasonable response length
)

print("✅ Chat Model initialized successfully!")
print(f"Model: {model.model_name}")
print(f"Temperature: {model.temperature}")
print(f"Max Tokens: {model.max_tokens}")

✅ Chat Model initialized successfully!
Model: gpt-4o-mini
Temperature: 0.7
Max Tokens: 1000


## 4. Defining the Chatbot State Schema

Chatbots need to maintain state across conversations. Let's define the structure for our chatbot's memory:

In [5]:
# Define the state schema for our chatbot
# This determines what information we track during conversations
# MessagesState is a LangGraph class that manages the state of the conversation and appends messages to the conversation history
class State(MessagesState):
    """The state of our chatbot conversation."""
    language: str

print("✅ Chatbot State Schema Defined!")

# State Structure:
# Messages: List of conversation messages
# Language: String for response language preference

✅ Chatbot State Schema Defined!


## 5. Creating the Prompt Template

The prompt template defines how our chatbot behaves and responds. Let's create a template that behaves as a helpful, professional business assistant.

In [6]:
# Create the prompt template for our chatbot
# This defines the personality and behavior of our chatbot
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional business assistant. "
              "You respond in {language}. "
              "Be concise, accurate, and maintain a professional tone. "
              "Always be helpful and provide value to the user."),
    MessagesPlaceholder(variable_name="messages"),
    ("human", "{input}")
])

print("✅ Chatbot Prompt Template Created!")

# Template Structure:
# - System Message: Defines chatbot personality and behavior
# - Messages Placeholder: Injects conversation history
# - Human Input: Captures current user message

✅ Chatbot Prompt Template Created!


## 6. Building the Chatbot Chain

Now let's create the core chatbot functionality by combining our prompt template with the language model:

In [7]:
# Create the chatbot chain
# This combines the prompt template with the language model
chain = prompt_template | model | StrOutputParser()

print("✅ Chatbot Chain Created!")

print("\nChain Components:")
print(f"1. Prompt Template: {type(prompt_template).__name__}")
print(f"2. Language Model: {type(model).__name__}")
print(f"3. Output Parser: {type(StrOutputParser()).__name__}")

# How It Works:
# - Prompt template formats the input with context
# - Chain invokes the language model with the formatted input
# - Output parser ensures consistent response format

✅ Chatbot Chain Created!

Chain Components:
1. Prompt Template: ChatPromptTemplate
2. Language Model: ChatOpenAI
3. Output Parser: StrOutputParser


## 7. Testing Basic Chatbot Functionality

Let's test our basic chatbot to ensure it's working correctly:

In [8]:
# Test the basic chatbot functionality
# This demonstrates the fundamental interaction pattern
try:
    # Simple test conversation
    test_messages = [
        HumanMessage(content="Hello! How can you help me today?")
    ]
    
    print("Testing Basic Chatbot:")
    print("="*50)
    
    response = chain.invoke({
        "messages": test_messages,
        "language": "English",
        "input": "Hello! How can you help me today?"
    })
    
    print(f"User: Hello! How can you help me today?")
    print(f"Chatbot: {response}")
    
    print("\n✅ Basic chatbot test successful!")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\nMake sure you have set your OpenAI API key correctly!")

Testing Basic Chatbot:
User: Hello! How can you help me today?
Chatbot: Hello! I can assist you with a variety of tasks, including providing information, answering questions, helping with business strategies, offering insights on industry trends, assisting with project management, and more. How can I assist you today?

✅ Basic chatbot test successful!


## 8. Introduction to LangGraph for Workflow Management

# <p align="center">
#   <img src="https://substackcdn.com/image/fetch/$s_!P6kf!,w_1200,h_600,c_fill,f_jpg,q_auto:good,fl_progressive:steep,g_auto/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fe0ef3bfd-a191-4fdd-aede-6e39c811ec8a_1920x1080.png" alt="LangGraph Workflow Example" style="max-width: 700px; width: 60%; height: auto;">
# </p>

Before we dive into implementing conversation memory, let's briefly introduce **LangGraph** - a powerful framework for building stateful, multi-actor applications with LLMs. We'll explore this technology in depth in future weeks, but for now, let's understand the core concepts we need for our chatbot.

### What is LangGraph?

LangGraph is a library for building stateful, multi-actor applications with LLMs. It's particularly powerful for:

- **State Management**: Maintaining conversation context and user information across interactions
- **Workflow Orchestration**: Managing complex conversation flows and decision trees
- **Multi-Agent Systems**: Coordinating multiple AI agents working together
- **Memory Persistence**: Storing and retrieving conversation history

### Key LangGraph Concepts

#### 1. StateGraph
A `StateGraph` is the core building block that defines how your application manages state. Think of it as a blueprint for your chatbot's memory and behavior:

```python
# StateGraph manages the flow of your application
workflow = StateGraph(state_schema=State)
```

#### 2. State Schema
The state schema defines what information your chatbot remembers:

```python
class State(MessagesState):
    """Defines what our chatbot remembers"""
    language: str  # User's preferred language
    # MessagesState automatically handles conversation history
```

#### 3. Nodes and Edges
- **Nodes**: Functions that process information (like generating responses)
- **Edges**: Define the flow between nodes (like conversation turns)

#### 4. Memory and Checkpointing
LangGraph provides built-in memory systems that persist conversation state:

```python
# MemorySaver stores conversation history
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
```

### Why LangGraph for Chatbots?

Traditional chatbot approaches often struggle with:
- **Context Loss**: Forgetting previous conversation turns
- **State Management**: Tracking user preferences and session data
- **Complex Flows**: Handling multi-step conversations and decision trees

LangGraph solves these challenges by providing:
- **Persistent State**: Automatic conversation history management
- **Workflow Control**: Clear definition of conversation flows
- **Memory Systems**: Built-in checkpointing and state persistence
- **Scalability**: Enterprise-ready architecture for production systems

### Enterprise Applications

In enterprise environments, LangGraph enables:
- **Customer Service Bots**: Multi-turn support conversations with context
- **Sales Automation**: Lead qualification with persistent user profiles
- **Internal Assistants**: Knowledge management with conversation history
- **Process Automation**: Complex workflows with state tracking

Now that we understand the basics, let's implement conversation memory using LangGraph:

In [9]:
# Create a StateGraph workflow for our chatbot with memory
# This enables multi-turn conversations and context preservation

# Step 1: Initialize the StateGraph with our state schema
# StateGraph is the core LangGraph component that manages application flow
workflow = StateGraph(state_schema=State)

# Step 2: Define the main chatbot processing function
# This function represents a "node" in our workflow graph
def call_model(state: State):
    """
    Process the current state and generate a response.
    
    This function:
    1. Receives the current conversation state
    2. Extracts the latest user message
    3. Generates an AI response using our chain
    4. Returns the updated state with the new AI message
    
    Args:
        state (State): Current conversation state containing messages and language preference
        
    Returns:
        dict: Updated state with new AI message appended
    """
    # Get the current input from the last human message
    current_input = state["messages"][-1].content if state["messages"] else ""
    
    # Generate response using our existing chain (prompt + model + parser)
    response = chain.invoke({
        "messages": state["messages"],
        "language": state["language"],
        "input": current_input
    })
    
    # Create AI message and add to conversation
    # MessagesState automatically handles message appending via reducers
    ai_message = AIMessage(content=response)
    
    # Return state update - LangGraph will merge this with existing state
    return {"messages": [ai_message]}

# Step 3: Build the workflow graph
# Add the processing node to our workflow
workflow.add_node("model", call_model)

# Define the conversation flow:
# START → model → END
# This creates a simple linear flow for our chatbot
workflow.add_edge(START, "model")  # Start conversation with model processing
workflow.add_edge("model", END)    # End after generating response

print("✅ Chatbot Workflow Created!")

✅ Chatbot Workflow Created!


In [20]:
print("\n🔧 Workflow Architecture:")
print("┌─────────┐    ┌─────────┐    ┌─────────┐")
print("│  START  │───▶│  MODEL  │───▶│   END   │")
print("└─────────┘    └─────────┘    └─────────┘")
print("   Input          Process        Output")

print("\n📋 Workflow Components:")
print(f"• State Schema: {State.__name__}")
print(f"• Processing Node: call_model()")
print(f"• Flow: Linear (START → MODEL → END)")
print(f"• Memory: MessagesState with automatic message management")

print("\n💡 Key Benefits:")
print("• Automatic state management")
print("• Conversation history preservation")
print("• Scalable workflow architecture")


🔧 Workflow Architecture:
┌─────────┐    ┌─────────┐    ┌─────────┐
│  START  │───▶│  MODEL  │───▶│   END   │
└─────────┘    └─────────┘    └─────────┘
   Input          Process        Output

📋 Workflow Components:
• State Schema: State
• Processing Node: call_model()
• Flow: Linear (START → MODEL → END)
• Memory: MessagesState with automatic message management

💡 Key Benefits:
• Automatic state management
• Conversation history preservation
• Scalable workflow architecture


In [11]:
# Compile the workflow with LangGraph's memory system
# This enables conversation persistence and state management

# Step 1: Create a memory checkpointer
# MemorySaver is LangGraph's built-in memory system that persists conversation state
memory = MemorySaver()

# Step 2: Compile the workflow with memory support
# The checkpointer parameter enables state persistence across conversation turns
app = workflow.compile(checkpointer=memory)

print("✅ Chatbot Application Compiled with Memory!")

print("\n🧠 LangGraph Memory System:")
print(f"Checkpointer Type: {type(memory).__name__}")

✅ Chatbot Application Compiled with Memory!

🧠 LangGraph Memory System:
Checkpointer Type: InMemorySaver


## 9. Testing LangGraph-Powered Multi-Turn Conversations

Now let's test our LangGraph-powered chatbot's ability to maintain conversation context and demonstrate the memory system in action:

In [12]:
# Test LangGraph-powered multi-turn conversation with memory
# This demonstrates the chatbot's ability to maintain context using LangGraph's state management

try:
    print("🧪 Testing LangGraph Multi-Turn Conversation:")
    print("="*60)
    
    # Set up configuration for this conversation thread
    # thread_id creates a unique conversation session
    config = {"configurable": {"thread_id": "langgraph_test_001"}}
    
    print("📋 Test Scenario: Name Memory Test")
    print("Goal: Verify that LangGraph maintains conversation context across turns")
    print()
    
    # First interaction - User introduces themselves
    print("🔁 Turn 1: User Introduction")
    print("-" * 40)
    messages = [HumanMessage(content="Hi! My name is Alice.")]
    language = "English"
    
    # Invoke the LangGraph application
    # LangGraph automatically manages state and memory
    output = app.invoke(
        {"messages": messages, "language": language},
        config
    )
    
    print(f"👤 User: Hi! My name is Alice.")
    print(f"🤖 Chatbot: {output['messages'][-1].content}")
    
    # Second interaction - Test memory retention
    print("\n🔁 Turn 2: Memory Test")
    print("-" * 40)
    query = "What is my name?"
    
    # For the second turn, we only need to provide the new message
    # LangGraph automatically retrieves the conversation history from memory
    new_messages = [HumanMessage(content=query)]
    
    output = app.invoke(
        {"messages": new_messages, "language": language},
        config
    )
    
    print(f"👤 User: What is my name?")
    print(f"🤖 Chatbot: {output['messages'][-1].content}")
    
    # Third interaction - Test continued context
    print("\n🔁 Turn 3: Extended Context Test")
    print("-" * 40)
    follow_up = "Can you help me with a business question?"
    
    output = app.invoke(
        {"messages": [HumanMessage(content=follow_up)], "language": language},
        config
    )
    
    print(f"👤 User: Can you help me with a business question?")
    print(f"🤖 Chatbot: {output['messages'][-1].content}")
    
    print("\n✅ LangGraph Multi-turn conversation test successful!")
    
    print("\n📝 Complete Conversation History:")
    for i, msg in enumerate(output['messages'], 1):
        msg_type = "👤 Human" if isinstance(msg, HumanMessage) else "🤖 AI"
        content_preview = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
        print(f"  {i}. {msg_type}: {content_preview}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("• Verify your OpenAI API key is correctly set")
    print("• Check that all dependencies are installed")
    print("• Ensure you have sufficient API credits")

🧪 Testing LangGraph Multi-Turn Conversation:
📋 Test Scenario: Name Memory Test
Goal: Verify that LangGraph maintains conversation context across turns

🔁 Turn 1: User Introduction
----------------------------------------
👤 User: Hi! My name is Alice.
🤖 Chatbot: Hello, Alice! How can I assist you today?

🔁 Turn 2: Memory Test
----------------------------------------
👤 User: What is my name?
🤖 Chatbot: Your name is Alice. How can I assist you further?

🔁 Turn 3: Extended Context Test
----------------------------------------
👤 User: Can you help me with a business question?
🤖 Chatbot: Absolutely! Please go ahead and ask your business question, and I'll do my best to assist you.

✅ LangGraph Multi-turn conversation test successful!

📝 Complete Conversation History:
  1. 👤 Human: Hi! My name is Alice.
  2. 🤖 AI: Hello, Alice! How can I assist you today?
  3. 👤 Human: What is my name?
  4. 🤖 AI: Your name is Alice. How can I assist you further?
  5. 👤 Human: Can you help me with a business q

In [13]:
print("\n🔍 LangGraph Memory Analysis:")
print("="*50)
print(f"📊 Total Messages in Memory: {len(output['messages'])}")
print(f"🧵 Thread ID: {config['configurable']['thread_id']}")
print(f"🌐 Language Preference: {output['language']}")


🔍 LangGraph Memory Analysis:
📊 Total Messages in Memory: 6
🧵 Thread ID: langgraph_test_001
🌐 Language Preference: English


## 10. Managing Conversation History

As conversations grow longer, we need to manage the message history to prevent context overflow. Let's implement message trimming:

In [14]:
# Import message trimming functionality
# This helps manage long conversations and prevent context overflow
from langchain_core.messages import trim_messages

# Create a message trimmer
# This ensures we don't exceed the model's context window
trimmer = trim_messages(
    max_tokens=100,  # Maximum tokens to keep
    strategy="last",   # Keep the most recent messages
    token_counter=model,  # Use our model's token counter
    include_system=True,  # Always keep system messages
    allow_partial=False,  # Don't allow partial messages
    start_on="human"     # Start trimming from human messages
)

In [15]:
# Demonstrate message trimming functionality
# This shows how the trimmer works with different message lengths

# Create a long conversation for demonstration
long_conversation = [
    SystemMessage(content="You are a helpful business assistant."),
    HumanMessage(content="Hello! I'm Bob."),
    AIMessage(content="Hello Bob! Nice to meet you."),
    HumanMessage(content="I like vanilla ice cream."),
    AIMessage(content="That's a classic choice! Vanilla is very popular."),
    HumanMessage(content="What's 2 + 2?"),
    AIMessage(content="2 + 2 equals 4."),
    HumanMessage(content="Thanks for the math help."),
    AIMessage(content="You're welcome! Math is fundamental."),
    HumanMessage(content="Having fun with our conversation?"),
    AIMessage(content="Yes, absolutely! I enjoy our chat."),
    HumanMessage(content="What's my name?")
]

In [16]:
print("📊 Message Trimming Demonstration:")
print("="*60)
print(f"Original Messages: {len(long_conversation)}")
print(f"Original Content: {sum(len(str(msg.content)) for msg in long_conversation)} characters")

📊 Message Trimming Demonstration:
Original Messages: 12
Original Content: 325 characters


In [17]:
# Apply trimming
trimmed_messages = trimmer.invoke(long_conversation)

print(f"\nTrimmed Messages: {len(trimmed_messages)}")
print(f"Trimmed Content: {sum(len(str(msg.content)) for msg in trimmed_messages)} characters")

print("\nRemaining Messages:")
for i, msg in enumerate(trimmed_messages):
    msg_type = type(msg).__name__
    content_preview = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
    print(f"{i+1}. {msg_type}: {content_preview}")

print("\n✅ Message trimming demonstration complete!")


Trimmed Messages: 8
Trimmed Content: 208 characters

Remaining Messages:
1. SystemMessage: You are a helpful business assistant.
2. HumanMessage: What's 2 + 2?
3. AIMessage: 2 + 2 equals 4.
4. HumanMessage: Thanks for the math help.
5. AIMessage: You're welcome! Math is fundamental.
6. HumanMessage: Having fun with our conversation?
7. AIMessage: Yes, absolutely! I enjoy our chat.
8. HumanMessage: What's my name?

✅ Message trimming demonstration complete!


## 11. Implementing Streaming Responses

Streaming is crucial for chatbot user experience. Let's implement streaming to show responses as they're generated:

In [19]:
# Test streaming functionality
# This demonstrates how responses can be streamed for better UX
try:
    print("🌊 Testing Streaming Responses:")
    print("="*50)
    
    # Set up configuration
    config = {"configurable": {"thread_id": "streaming_test_001"}}
    
    # Create a test message
    query = "Hi I'm Todd, please tell me a joke."
    language = "English"
    input_messages = [HumanMessage(content=query)]
    
    print(f"User: {query}")
    print("Chatbot (streaming): ", end="")
    
    # Stream the response
    for chunk, metadata in app.stream(
        {"messages": input_messages, "language": language},
        config,
        stream_mode="messages"
    ):
        if isinstance(chunk, AIMessage):
            print(chunk.content, end="")
    
    print("\n\n✅ Streaming test successful!")
    print("\🎥 Streaming Benefits:")
    print("- Better user experience")
    print("- Perceived faster responses")
    print("- Real-time interaction")
    print("- Reduced wait time perception")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\nMake sure you have set your OpenAI API key correctly!")

🌊 Testing Streaming Responses:
User: Hi I'm Todd, please tell me a joke.
Chatbot (streaming): Hi Todd! Here’s a joke for you: 

Why did the scarecrow win an award? 

Because he was outstanding in his field! 

I hope that brings a smile to your day!Hi Todd! Here’s a joke for you: 

Why did the scarecrow win an award? 

Because he was outstanding in his field! 

I hope that brings a smile to your day!

✅ Streaming test successful!
\🎥 Streaming Benefits:
- Better user experience
- Perceived faster responses
- Real-time interaction
- Reduced wait time perception


## 12. Key Takeaways

### What We've Accomplished

1. **Basic Chatbot Creation**: Established the fundamental pattern for building chatbots
2. **Conversation Memory**: Implemented state management and context preservation
3. **Message History Management**: Created systems for handling long conversations
4. **Streaming Responses**: Implemented real-time response generation

### Architecture Considerations

#### Scalability
- Chatbots can be deployed as microservices
- Memory systems can be distributed across multiple instances

#### Security
- Conversation history must comply with data privacy regulations
- User authentication and authorization must be implemented
- Sensitive information should not be stored in conversation logs

#### Integration
- Chatbots can integrate with existing CRM and business systems
- Message history can be used for analytics and improvement

#### Cost Management
- Token usage should be monitored and optimized
- Message trimming helps control costs
- Streaming can improve perceived performance without increasing costs

### Common Enterprise Use Cases

- **Customer Service**: 24/7 support with human escalation
- **Sales Support**: Lead qualification and follow-up automation
- **Technical Support**: First-level troubleshooting and ticket creation
- **Internal Knowledge**: Employee self-service and information retrieval
- **Training and Onboarding**: Interactive learning and guidance

## 13. Exercise: Build Your First Chatbot

Now it's your turn to create a specialized chatbot for a specific business scenario. Choose one of the following exercises:

In [ ]:
# Exercise 1: HR Assistant Chatbot
# Create a chatbot that can help employees with HR-related questions

def create_hr_assistant():
    """Create an HR assistant chatbot."""
    
    # TODO: Implement your HR assistant chatbot here
    # 1. Create a system prompt for HR assistance
    # 2. Define HR-specific conversation flows
    # 3. Implement appropriate responses for common HR questions
    # 4. Test with sample HR scenarios
    
    print("👥 Exercise 1: HR Assistant Chatbot")
    print("Create a chatbot that can:")
    print("- Answer common HR questions")
    print("- Provide policy information")
    print("- Guide employees through processes")
    print("- Maintain professional HR standards")
    
    return "HR assistant implementation goes here"

# Exercise 2: Product Support Chatbot
# Create a chatbot that can help customers with product-related questions

def create_product_support():
    """Create a product support chatbot."""
    
    # TODO: Implement your product support chatbot here
    # 1. Create a system prompt for product support
    # 2. Define product knowledge base integration
    # 3. Implement troubleshooting workflows
    # 4. Test with sample product issues
    
    print("🔧 Exercise 2: Product Support Chatbot")
    print("Create a chatbot that can:")
    print("- Answer product questions")
    print("- Guide troubleshooting steps")
    print("- Provide product information")
    print("- Escalate complex issues")
    
    return "Product support chatbot implementation goes here"

# Exercise 3: Training and Onboarding Chatbot
# Create a chatbot that can help new employees with onboarding

def create_onboarding_assistant():
    """Create an onboarding assistant chatbot."""
    
    # TODO: Implement your onboarding assistant chatbot here
    # 1. Create a system prompt for onboarding assistance
    # 2. Define onboarding workflows and checklists
    # 3. Implement progress tracking
    # 4. Test with sample onboarding scenarios
    
    print("🎓 Exercise 3: Training and Onboarding Chatbot")
    print("Create a chatbot that can:")
    print("- Guide new employees through onboarding")
    print("- Provide training resources")
    print("- Track progress and completion")
    print("- Answer common new employee questions")
    
    return "Onboarding assistant implementation goes here"

print("🚀 Ready to build your first chatbot!")

## 14. Conclusion

Congratulations! You've successfully completed the chatbot development lesson with LangChain. You now understand:

### Technical Competencies
- How to create functional chatbots with conversation memory
- How to implement state management and context preservation
- How to manage conversation history and prevent context overflow
- How to implement streaming responses for better user experience
- How to customize chatbots for different business roles

### Insights
- The architectural implications of chatbot systems
- How chatbots can transform customer service and business processes
- Common use cases and implementation patterns

### Next Steps

In the next lesson, you'll learn about:
- Building AI agents with tools and capabilities

---

Happy coding! 🚀